In [182]:
# !pip install --upgrade pip
# !pip install bs4 selenium black pandas --quiet

In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import re
import pandas as pd
import time
from bs4 import BeautifulSoup
from datetime import datetime

In [6]:
sites = {
    "yahoo": {
        "base_url": "https://finance.yahoo.com/topic/",
        "category": {
            "economics": "economic-news",
            "tech": "tech",
            "housing": "housing-market",
        },
    }
}

In [10]:
request_headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36 Edg/125.0.0.0",
    "Referer": "",
}

In [11]:
req = requests.get("https://finance.yahoo.com/topic/economic-news/")
soup = BeautifulSoup(req.content, 'html.parser')
# article_titles = [t.text for t in soup.find_all(class_="Mb(5px)")]
# links = [t.find('a')["href"] for t in soup.find_all(class_="Mb(5px)")]
titles = soup.find_all(class_="Mb(5px)")
title_text = [t.text for t in titles]
links = [t.find("a")["href"] for t in titles]

In [12]:
len(titles)

2

In [13]:
links

['https://finance.yahoo.com/news/asian-stocks-drop-traders-rethink-224525599.html',
 'https://finance.yahoo.com/news/oil-set-eke-weekly-gain-232819911.html',
 'https://finance.yahoo.com/news/oil-set-weekly-gain-signs-010613579.html',
 'https://finance.yahoo.com/news/suriname-oil-resources-seen-totaling-223930531.html',
 'https://finance.yahoo.com/news/fed-mester-suggests-interest-rates-160000693.html',
 'https://finance.yahoo.com/news/petro-meets-colombian-central-bankers-220147115.html',
 'https://finance.yahoo.com/news/firefighters-gain-control-one-blaze-195526122.html',
 'https://finance.yahoo.com/news/oil-ticks-higher-us-stockpile-232703377.html',
 'https://finance.yahoo.com/news/us-proposes-ending-federal-leases-174509228.html',
 'https://finance.yahoo.com/news/banks-continue-become-increasingly-less-173404596.html']

In [ ]:
def parse_articles_bs(link):
    req = requests.get(link)
    soup = BeautifulSoup(req.content, 'html.parser')    
    title_text = [t.text for t in soup.find_all(class_="Mb(5px)")]
    links = [t.find('a')["href"] for t in soup.find_all(class_="Mb(5px)")]
    articles = [get_article(l) for l in links]
    
    df = pd.DataFrame({'title': title_text, 'link': links, 'content': articles})
    return df

In [178]:
def parse_articles_selenium(article_list):
    titles = article_list.find_elements(by=By.CLASS_NAME, value="sa_text")
    title_text = [t.find_element(by=By.CLASS_NAME, value="sa_text_title").text for t in titles]
    links = [t.find_element(by=By.CLASS_NAME, value="sa_text_title").get_attribute("href") for t in titles]
    articles = [get_article(l) for l in links]
    
    df = pd.DataFrame({'title': title_text, 'link': links, 'content': articles})
    return df

In [15]:
def get_article(link: str):
    req = requests.get(link)
    soup = BeautifulSoup(req.content, 'html.parser')
    date = soup.select_one("#caas-art-9dffa534-bcd1-3e24-8377-ba1fb9d5347e > article > div > div > div > div > div > div > div:nth-child(1) > div.caas-content-byline-wrapper > div.caas-attr > div > div.caas-attr-time-style > time")["datetime"]
    content = '\n'.join([p.text for p in soup.find(class_="caas-body").select("p")])
    return {"date": date, "content": content}

In [201]:
req = requests.get("https://news.naver.com/breakingnews/section/101/259")
soup = BeautifulSoup(req.content, 'html.parser')
titles = [t.text.strip('\n') for t in soup.find_all(class_="sa_text_title")]
links = [t['href'] for t in soup.find_all(class_="sa_text_title")]

In [210]:
datetime.date()

datetime.datetime(2024, 5, 17, 15, 59, 24, 340080)

In [32]:
request_headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36 Edg/125.0.0.0",
    "Referer": "",
}

In [33]:
link = 'https://finance.yahoo.com/news/york-lawmakers-pass-237-billion-203737865.html'
req = requests.get(link, headers=request_headers)
soup = BeautifulSoup(req.content, 'html.parser')
# date = soup.select_one("time")["datetime"]
# content = '\n'.join([p.text for p in soup.find(class_="caas-body").select("p")])

In [36]:
date = soup.find("time")
date

<time class="caas-attr-meta-time" datetime="2024-04-21T00:52:43.000Z">Sun, Apr 21, 2024, 9:52 AM</time>

In [35]:
soup.text


'New York lawmakers pass $237 billion budget with policies to jump-start housing market           News Today\'s news  US  Politics  World  Tech  Reviews and deals  Audio  Computing  Gaming  Health  Home  Phones  Science  TVs    Climate change  Health  Science  2024 election  Originals  The 360      Life Health  COVID-19  Fall allergies  Health news  Mental health  Relax  Sexual health  Studies  The Unwind    Parenting  Family health  So mini ways    Style and beauty  It Figures  Unapologetically    Horoscopes  Shopping  Buying guides    Food  Travel  Autos  Gift ideas  Buying guides    Entertainment Celebrity  TV  Movies  Music  How to Watch  Interviews  Videos  Shopping     Finance My Portfolio  My Portfolio    News  Latest News  Stock Market  Originals  Premium News  Economics  Housing  Earnings  Tech  Crypto  Biden Economy  EV Deep Dive    Markets  Stocks: Most Actives  Stocks: Gainers  Stocks: Losers  Calendars  Trending Tickers  Futures  World Indices  US Treasury Bonds  Currencie

In [29]:
req.url

'https://finance.yahoo.com/news/york-lawmakers-pass-237-billion-203737865.html'